In [1]:
# Variáveis Iniciais do Sistema
initial_date = "20-02-2020"
end_date = "20-05-2020"
total_registers = 900
total_users = 1200
total_locations = 5

initial_time = "08:00"
end_time = "23:00"

# Dois tipos de veiculos -> 90% serao carros e 10% serao bicicletas.
use_type = False
vehicle_types = { 'car': 0.9, 'bike': 0.1 }

# Dois tipos de clima -> em dias de chuva tem aumento de 50% na demanda comum.
use_weather = False
impact_weather = { 'sunny': { 'impact': 0, 'chance': 0.95 }, 'rain': { 'impact': 0.5, 'chance': 0.05 } }

# Definir todas as datas como feriado ou nao -> em dias de feriado tem diminuiçãod e 35% na demanda comum.
use_holiday = False
impact_holiday = -0.35
holidays = ["25-12","31-12"]

In [2]:
# Importações de Bibliotecas Necessárias
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import timedelta  

from random import seed
from random import randint

from matplotlib import pyplot as plt
import matplotlib.dates as md

In [3]:
#cols = ["date","distanceRange","locationWeight","maxPrice","parking","parkingName","priceWeight",
#        "region","spotWanted","timeFrom","timeTo","vehicleId","vehicleType"]

cols = ["timeFrom", "timeTo"]

open_time = datetime.strptime(initial_time, '%H:%M')
close_time = datetime.strptime(end_time, '%H:%M')

date = datetime.strptime(initial_date, '%d-%m-%Y')
date = date.replace(hour=open_time.hour, minute=open_time.minute)

end_date = datetime.strptime(end_date, '%d-%m-%Y')

current_parking = []

#date = date + timedelta(minutes=60*13)
#date = date + timedelta(minutes=60*11)

print(date.strftime('%d-%m-%Y'))
print(open_time.strftime('%H:%M'))
print(close_time.strftime('%H:%M'))
date

20-02-2020
08:00
23:00


datetime.datetime(2020, 2, 20, 8, 0)

In [9]:
df = pd.DataFrame(columns=cols)

print("Generating Data...")

while(date < end_date):
    
    # Cria uma ROW
    data = {}
    data.update({'timeFrom': date.strftime('%d-%m-%Y %H:%M')})
    
    # Trecho que define o tempo de estacionamento e captura dados do usuario
    # ------
    
    parking_time = randint(15,160)
    validate_date = date + timedelta(minutes=parking_time)
    
    if (validate_date.hour >= close_time.hour):
        data.update({'timeTo': (date.replace(hour=close_time.hour, minute=close_time.minute).strftime('%d-%m-%Y %H:%M')) })

    elif (validate_date.hour < open_time.hour):
        data.update({'timeTo': (date.replace(hour=close_time.hour, minute=close_time.minute).strftime('%d-%m-%Y %H:%M')) })
        
    else:
        data.update({'timeTo': validate_date.strftime('%d-%m-%Y %H:%M')})
       
    # ------
    
    df = df.append(data, ignore_index=True)
    
    
    # 20% chance de adicionar minutos
    isAddMin = randint(0,5)
    if (isAddMin >= 4):
        addMin = randint(1, 15)
        date = date + timedelta(minutes=addMin)
        
        if (date.hour >= close_time.hour):
            date = date + timedelta(days=1)
            date = date.replace(hour=open_time.hour, minute=open_time.minute)
        elif( (date.hour == close_time.hour-1) and (60 - date.minute <= 15)):
            date = date + timedelta(days=1)
            date = date.replace(hour=open_time.hour, minute=open_time.minute)
        elif (date.hour < open_time.hour):
            date = date.replace(hour=open_time.hour, minute=open_time.minute)
    
print("Done!")

Generating Data...
Done!


In [10]:
df

,timeFrom,timeTo
0,29-02-2020 23:13,29-02-2020 23:00
1,29-02-2020 23:13,29-02-2020 23:00
2,01-03-2020 08:00,01-03-2020 08:49
3,01-03-2020 08:11,01-03-2020 08:30
4,01-03-2020 08:11,01-03-2020 09:54
...,...,...
26353,19-05-2020 22:25,19-05-2020 23:00
26354,19-05-2020 22:25,19-05-2020 23:00
26355,19-05-2020 22:32,19-05-2020 23:00
26356,19-05-2020 22:40,19-05-2020 23:00


In [11]:
df.to_csv('generated_dataset.csv', index=False)

In [ ]:
print("Setting the Pattern in DateTime Columns...")
df['timeFrom'] = pd.to_datetime(df['timeFrom'], errors='ignore')
df['timeFrom'] = pd.to_datetime(df["timeFrom"].dt.strftime('%d/%m/%Y %H:%M'))

df['timeTo'] = pd.to_datetime(df['timeTo'], errors='ignore')
df['timeTo'] = pd.to_datetime(df["timeTo"].dt.strftime('%d/%m/%Y %H:%M'))

# Query For Specific Day
print("Setting Query for Specific Day and Month...")
df = df[(df['timeFrom'].dt.month == 2) & (df['timeFrom'].dt.day == 20)]

print("Generating Graph...")
fig, ax = plt.subplots(figsize=(15,15))

#min_y = 0
max_y = 0
#max_x = 0
min_x = 0

#while ((df['timeTo'].min().minute - min_y) % 30 != 0):
#    min_y -= 1
    
while ((df['timeTo'].max().minute + max_y) % 30 != 0):
    max_y += 1
    
while ((df['timeFrom'].min().minute + min_x) % 30 != 0):
    min_x -= 1

min_x = min_x + 30
max_y = max_y + 30
    
#while ((df['timeFrom'].max().minute + max_x) % 30 != 0):
#    max_x += 1
    
    
plt.scatter(x=df['timeFrom'], y=df['timeTo'])

ax.set_xlim(df['timeFrom'].min()-pd.Timedelta(min_x,'m'),
            df['timeTo'].max()+pd.Timedelta(max_y,'m'))

ax.set_ylim(df['timeFrom'].min()-pd.Timedelta(min_x,'m'),
            df['timeTo'].max()+pd.Timedelta(max_y,'m'))

ax.xaxis.set_major_locator(md.MinuteLocator(interval = 30))
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))

ax.yaxis.set_major_locator(md.MinuteLocator(interval = 30))
ax.yaxis.set_major_formatter(md.DateFormatter('%H:%M'))

fig.autofmt_xdate()

plt.title("\nTotal: "+str(len(df['timeTo'])), fontsize=30, pad=20)
plt.xlabel("Enter DateTime", fontsize=20, labelpad=20)
plt.ylabel("Exit DateTime", fontsize=20, labelpad=20)

plt.xticks(rotation=90)

plt.grid()
plt.show()